In [ ]:
import sys
import os
import pickle
import logging
import numba
import matplotlib
import matplotlib.pyplot as plt
logging.getLogger('matplotlib.font_manager').disabled = True
numba_logger = logging.getLogger('numba')
numba_logger.setLevel(logging.WARNING)

matplotlib_logger = logging.getLogger('matplotlib')
matplotlib_logger.setLevel(logging.WARNING)

# Add the src directory to sys.path
src_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
print(src_path)
sys.path.append(src_path)

from src.IndependentSteps import Pycromanager2NativeDataType, FFF2NativeDataType

from src.SequentialSteps import BIGFISH_SpotDetection, SimpleCellposeSegmentaion, IlluminationCorrection, \
                                Calculate_BIGFISH_Threshold, CellProperties, Automatic_BIGFISH_Threshold

from src.FinalizationSteps import Save_Outputs, Save_Images, Save_Parameters, Save_Masks, return_to_NAS, remove_local_data_but_keep_h5, \
                                     remove_local_data

from src.Parameters import Parameters, Experiment, Settings, ScopeClass, DataContainer

from src.Displays import Display

from src.GUI import GUI, StepGUI

from src.Pipeline import Pipeline

In [ ]:
def runner(idl, ip):
    # Initialize Parameters
    scope = ScopeClass()
    scope.voxel_size_yx = 160
    scope.spot_yx = 300
    scope.spot_z = 500 
    data = DataContainer()
    data.clear_after_error = False
    settings = Settings(name='ER_Dec0324_DUSP1', ) 
    experiment = Experiment()
    experiment.independent_params = ip
    experiment.initial_data_location = idl
    # experiment.snr_threshold = 4 # were going to do this post processing

    settings.load_in_mask = False
    experiment.FISHChannel = 0
    experiment.nucChannel = 2
    experiment.cytoChannel = 1
    experiment.voxel_size_z = 500

    settings.cellpose_min_size = 100
    settings.cellpose_diameter = [180, 90]
    settings.cellpose_pretrained_model = ["GAPDH_cyto", 'DAPI_nuclei'] 

    settings.bigfish_threshold = 'mean'
    settings.verbose = True
    settings.MAX_NUM_IMAGES_TO_AUTOMATICALLY_CALCULATE_THRESHOLD = 1_000_000

    FFF2NativeDataType()
    Automatic_BIGFISH_Threshold()
    SimpleCellposeSegmentaion()
    BIGFISH_SpotDetection()
    CellProperties() # this is probably not need but oh well
    Save_Masks()
    Save_Parameters()
    Save_Outputs()
    return_to_NAS()
    remove_local_data()

    Parameters.validate()

    pipeline = Pipeline()

    pipeline.run_on_cluster('/home/formanj/Github/AngelFISH/cluster', name=settings.name) # give this a slightluy better name the date is already saved 

    pipeline.clear_pipeline() # I dont think this is nesscary but its probablily good to do
    

In [ ]:
# List of directories to process
initial_data_location='smFISH_images/Eric_smFISH_images/20230316/DUSP1_10nM_Dex_30min_012623'
        # 'smFISH_images/Eric_smFISH_images/20230306/DUSP1_0nM_Dex_0min_012623' ,
        # 'smFISH_images/Eric_smFISH_images/20230306/DUSP1_300pM_Dex_30min_012623' ,
        # 'smFISH_images/Eric_smFISH_images/20230306/DUSP1_300pM_Dex_50min_012623' ,
        # 'smFISH_images/Eric_smFISH_images/20230306/DUSP1_300pM_Dex_75min_012623' ,
        # 'smFISH_images/Eric_smFISH_images/20230309/DUSP1_300pM_Dex_90min_012623' ,
        # 'smFISH_images/Eric_smFISH_images/20230309/DUSP1_300pM_Dex_120min_012623' ,
        # 'smFISH_images/Eric_smFISH_images/20230309/DUSP1_300pM_Dex_180min_012623' ,
        # 'smFISH_images/Eric_smFISH_images/20230309/DUSP1_1nM_Dex_30min_012623' ,
        # 'smFISH_images/Eric_smFISH_images/20230310/DUSP1_1nM_Dex_50min_012623' ,
        # 'smFISH_images/Eric_smFISH_images/20230310/DUSP1_1nM_Dex_75min_012623' ,
        # 'smFISH_images/Eric_smFISH_images/20230310/DUSP1_1nM_Dex_90min_012623' ,
        # 'smFISH_images/Eric_smFISH_images/20230310/DUSP1_1nM_Dex_120min_012623' ,
        # 'smFISH_images/Eric_smFISH_images/20230316/DUSP1_1nM_Dex_180min_012623' ,
        # 'smFISH_images/Eric_smFISH_images/20230316/DUSP1_10nM_Dex_30min_012623' ,
        # 'smFISH_images/Eric_smFISH_images/20230316/DUSP1_10nM_Dex_50min_012623' ,
        # 'smFISH_images/Eric_smFISH_images/20230316/DUSP1_10nM_Dex_75min_012623' ,
        # 'smFISH_images/Eric_smFISH_images/20230316/DUSP1_10nM_Dex_90min_012623' ,
        # 'smFISH_images/Eric_smFISH_images/20230316/DUSP1_10nM_Dex_120min_012623' ,
        # 'smFISH_images/Eric_smFISH_images/20230316/DUSP1_10nM_Dex_180min_012623' )
# Parameters
Condition = 'DUSP1_timesweep'      # Experimental condition. The options are:  'GR_timesweep' , 'DUSP1_timesweep' 
Replica = 'J'                   # Replica name. 
time_list  = [0, 30, 50, 75, 90, 120, 180, 30, 50, 75, 90, 120, 180, 30, 50, 75, 90, 120, 180]        # Time of image acquisition
DexConc_list = [0, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 1, 1, 1, 1, 1, 1, 10, 10, 10, 10, 10, 10]        # Dex concentration

# Creating the list of dictionaries
DUSP1_TCS_R1 = [
    {
        "condition": Condition,
        "replica": Replica,
        "time": time,
        "Dex_Conc": DexConc
    }
    for time, DexConc in zip(time_list, DexConc_list)
]
# Output the result
print(DUSP1_TCS_R1)

In [ ]:
runner(initial_data_location, DUSP1_TCS_R1)